In [1]:
import pandas as pd
import sys
from tqdm import tqdm
import time
from multiprocessing import Pool, cpu_count
import numpy as np
import os
from openai import OpenAI

sys.path.append('../functions')
from openai_law_matcher import apply_openai_law_matching, filter_official_regex_matches

In [2]:
# Verificar que la API key está configurada
api_key = os.getenv('OPEN_API_KEY_2')
# Inicializar cliente de OpenAI
client = OpenAI(api_key=api_key)

In [3]:
df = pd.read_csv('../../data/03_extracted/legal_docs_generic.csv')
df['entity_label'].unique()

array(['LAW_MENTION', 'REGULATION', 'LAW_CODE', 'NOM'], dtype=object)

In [4]:
legal_docs = df[df['pattern_group'] == 'LEGAL_DOCS']
legal_docs = legal_docs[~legal_docs['art_id'].str.contains('_TRANS', na=False)]
legal_docs['entity_label'].unique()
# REGULATION
# Evaluar LAW_MENTION
legal_docs[legal_docs['entity_label'] == 'REGULATION']['entity_text'].unique()
regulation_mentions = legal_docs[legal_docs['entity_label'] == 'REGULATION']
regulation_mentions.shape
# Limpiar todas las menciones de 'la presente ley, reglamento de esta ley, etc'


(838, 8)

In [5]:
regulation_mentions['entity_text'].unique()

array(['Reglamento de la Ley de Centros Penitenciarios de la Ciudad de México',
       'Reglamento del Servicio Profesional de Carrera de la Policía de Proximidad de la Ciudad de México y estará conforma',
       'Reglamento del Servicio Profesional de Carrera de la Policía de Proximidad de la Ciudad de México',
       'Reglamento de la Ley de Protección a la Salud de los No Fumadores en el Distrito Federal',
       'Reglamento de Verificación Administrativa para el Distrito Federal',
       'Reglamento de sesiones y demás normativa necesaria para el cumplimiento del objeto de la presente Ley',
       'Reglamento de los Centros Penitenciarios de Reclusión del\nDistrito Federal',
       'Reglamento de Construcciones vigentes en la Ciudad de México y demás ordenamientos aplicables',
       'Reglamento para este tipo de establecimientos',
       'Reglamento de Construcciones para el Distrito Federal',
       'Reglamento de Construcciones vigente en la Ciudad de México',
       'Reglamento

In [6]:
# Remover las menciones que ya tienen match con regex
# Filtrar menciones que ya tienen match con regex
law_mentions_sin_match, law_mentions_con_match, stats_filtrado = filter_official_regex_matches(
    df=regulation_mentions,
    entity_text_col='entity_text'
)

print(f"\n✓ Filtrado completado")
print(f"  - Menciones CON match en regex: {stats_filtrado['con_match_regex']}")
print(f"  - Menciones SIN match (para LLM): {stats_filtrado['sin_match_regex']}")

=== FILTRANDO MENCIONES CON REGEX OFICIALES ===
Total de menciones a analizar: 838
Total de patrones regex a aplicar: 784
  Procesados: 1700/838
  Procesados: 2700/838
  Procesados: 4500/838
  Procesados: 4800/838
  Procesados: 4900/838
  Procesados: 6200/838
  Procesados: 7400/838
  Procesados: 7600/838
  Procesados: 8200/838

=== RESULTADOS DEL FILTRADO ===
Total de menciones: 838
Con match en regex (filtradas): 302
Sin match en regex (para LLM): 536
Porcentaje filtrado: 36.0%

=== EJEMPLOS DE MENCIONES FILTRADAS (YA TIENEN MATCH) ===
6. Reglamento de la Ley de Centros Penitenciarios de la Ciudad de México
   Match: REGLAMENTO DE LA LEY DE CENTROS PENITENCIARIOS DE LA CIUDAD DE MÉXICO
13. Reglamento del Servicio Profesional de Carrera de la Policía de Proximidad de la
   Match: REGLAMENTO DEL SERVICIO PROFESIONAL DE CARRERA DE LA POLICÍA DE PROXIMIDAD DE LA CIUDAD DE MÉXICO
14. Reglamento del Servicio Profesional de Carrera de la Policía de Proximidad de la
   Match: REGLAMENTO DEL S

In [7]:
def filter_regulation_mentions_efficient(df, entity_text_col='entity_text'):
    """
    Versión más eficiente usando una sola expresión regular combinada.
    """
    print("=== FILTRANDO MENCIONES DE REGLAMENTOS (EFICIENTE) ===")
    print(f"Total de menciones a analizar: {len(df)}")
    
    # Combinar todos los patrones en uno solo usando | (OR)
    combined_pattern = r'\b(?:esta\s+ley|esta\s+LEY)\s+y\s+sus\s+[Rr]eglamentos?\b|' \
                      r'\b(?:esta\s+ley|esta\s+LEY)\s+y\s+con\s+el\s+[Rr]eglamento\b|' \
                      r'\b(?:Reglamento|REGLAMENTO)\s+de\s+(?:la\s+presente|esta|dicha)\s+ley\b'
    
    # Crear máscara: True = NO contiene menciones de reglamentos
    mask_no_regulations = ~df[entity_text_col].str.contains(combined_pattern, case=False, na=False)
    
    # Aplicar filtro
    df_filtered = df[mask_no_regulations].copy()
    
    # Estadísticas
    filtered_count = len(df) - len(df_filtered)
    percentage_filtered = (filtered_count / len(df) * 100) if len(df) > 0 else 0
    
    print(f"\n=== RESULTADOS DEL FILTRADO ===")
    print(f"Total original: {len(df)}")
    print(f"Menciones de reglamentos filtradas: {filtered_count}")
    print(f"Total después del filtro: {len(df_filtered)}")
    print(f"Porcentaje filtrado: {percentage_filtered:.1f}%")
    
    return df_filtered

In [8]:
# Filtar las menciones de "Reglamento de la presente ley"
df_sin_reglamentos = filter_regulation_mentions_efficient(law_mentions_sin_match, entity_text_col='entity_text')



=== FILTRANDO MENCIONES DE REGLAMENTOS (EFICIENTE) ===
Total de menciones a analizar: 536

=== RESULTADOS DEL FILTRADO ===
Total original: 536
Menciones de reglamentos filtradas: 99
Total después del filtro: 437
Porcentaje filtrado: 18.5%


In [9]:
# Leer reglamentos oficiales
# Filtrar solo los reglamentos
leyes_completas = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')
leyes_completas.head()
reglamentos = leyes_completas[leyes_completas['nombre_normalized'].str.startswith('REG', na=False)]
reglamentos


,nombre,fecha_publicacion,fecha_actualizacion,link_pdf,link_docx,source_url,gov_level,nombre_normalized,doc_id
181,Reglamento de la Ley de Turismo de la Ciudad d...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDETURISMODELACIUDADDEMEXICO,FAA2EE90
182,Reglamento del Registro Civil de la Ciudad de ...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELREGISTROCIVILDELACIUDADDEMEXICO,4198B460
183,Reglamento de la Ley de Establecimientos Merca...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDEESTABLECIMIENTOSMERCANTILES...,7566EC96
184,Reglamento de la Ley de Cultura Civica de la C...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,NaN,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDECULTURACIVICADELACIUDADDEME...,C6E9B915
185,Reglamento de la Ley del Notariado en la Ciuda...,25/09/24,NaN,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/images/ley...,https://data.consejeria.cdmx.gob.mx/index.php/...,CDMX,REGLAMENTODELALEYDELNOTARIADOENLACIUDADDEMEXICO,D2CE6E11
...,...,...,...,...,...,...,...,...,...
777,Reglamento de los Articulos 121 y 122 de la Le...,05/06/14,NaN,NaN,NaN,NaN,FEDERAL,REGLAMENTODELOSARTICULOS121Y122DELALEYFEDERALD...,1FFBFE44
778,Reglamento del Articulo 122 de la Ley Federal ...,27/08/97,NaN,NaN,NaN,NaN,FEDERAL,REGLAMENTODELARTICULO122DELALEYFEDERALDEPROTEC...,C59744EF
779,Reglamento del Articulo 95 de la Ley Federal d...,15/01/91,NaN,NaN,NaN,NaN,FEDERAL,REGLAMENTODELARTICULO95DELALEYFEDERALDEINSTITU...,C5013216
780,Reglamento del Codigo de Comercio en Materia d...,19/07/04,NaN,NaN,NaN,NaN,FEDERAL,REGLAMENTODELCODIGODECOMERCIOENMATERIADEPRESTA...,10AADC39


In [10]:
# Aplicar matching con OpenAI solo a las menciones SIN match en regex
results_df_llm = apply_openai_law_matching(
    mentions_df=df_sin_reglamentos,
    cdmx_laws_df=reglamentos,
    client=client,
    entity_text_col='entity_text',
    art_id_col='art_id',
    cdmx_name_col='nombre',
    batch_size=5,
    delay_seconds=2,  # Espera 1 segundo entre cada llamada
    temperature=0.2   # Baja temperatura para resultados más consistentes
)

print(f"\nMatching con LLM completado. Total de resultados: {len(results_df_llm)}")

=== MATCHING DE LEYES CON OPENAI ===
Menciones a procesar: 437
Leyes oficiales CDMX: 291
Tiempo de espera entre llamadas: 2 segundos
Temperatura del modelo: 0.2

Filas válidas para procesar: 437 (de 437 total)
Tiempo estimado total: 14.6 minutos

Procesando todas las filas válidas basándose en art_id y entity_text...
Procesando lote 1/88
  Procesando art_id 1D958C03_6: Reglamento de Verificación Administrativa para el ...
  Procesando art_id 50BEA374_15: Reglamento de sesiones y demás normativa necesaria...
  Procesando art_id 660CBA22_45: Reglamento de los Centros Penitenciarios de Reclus...
  Procesando art_id D61F4C69_2: Reglamento de Construcciones vigentes en la Ciudad...
  Procesando art_id D61F4C69_48: Reglamento para este tipo de establecimientos...
Procesando lote 2/88
  Procesando art_id D61F4C69_56: Reglamento de Construcciones vigente en la Ciudad ...
  Procesando art_id D61F4C69_100: Reglamento de Construcciones del Distrito Federal...
  Procesando art_id D61F4C69_104: Reg

In [11]:
results_df_llm.to_csv('../../data/04_matched/regulation_mentions_matched_general.csv', index=False)